In [21]:
import pandas as pd
from IPython.display import HTML, display
from os.path import join as pjoin
import csv
import pickle
import math
import heapq


#m = pd.read_csv("Airbnb_Texas_Rentals.csv",encoding = 'utf8')

In [22]:
#we found words like '\\n' in the dataset so we cleaned it
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\t',  ' ', regex=True)
#m = m.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\r',  ' ', regex=True)

In [23]:
#create tsv files,we droped the first column that was a prroblem
#m = m.drop(['Unnamed: 0'], axis=1)

In [24]:
#we create the vocabulary of preprocessed documents,but we don't modify the documents because we''l use them in search engine
import nltk
import csv
import re
import os
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+|\$')
ps = SnowballStemmer('english')

review_content_persist = {}
review_word_map_persist = {}
my_path = os.path.dirname(os.path.realpath('__file__'))


# Retrieving persisted information for review content and word map
with open(os.path.join(my_path, "indexes/review_content.pkl"), "rb") as review_content:
    review_content_persist = pickle.load(review_content)
    review_content.close()
    
with open(os.path.join(my_path, "indexes/review_word_map.pkl"), "rb") as review_word_map:
    review_word_map_persist = pickle.load(review_word_map)
    review_word_map.close()

if(len(review_word_map_persist.keys()) == 0):
    
    print("Indexes are being created")
    
    for i in range(18259):
        with open(os.path.join(my_path, 'docu_hw3/doc_' + str(i) + '.tsv'),encoding='utf8') as tsvfile:
             tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
        l1 = tsvreader[0][4]
        l2 = tsvreader[0][7]
        l = l1+ ' ' +l2
        l = l.lower()
        l = tokenizer.tokenize(l)
        
        file_words = []
        
        for r in l :
            if not r in stop_words:
                sr = ps.stem(r)
                
                file_words.append(sr)
                
                if not  sr in review_word_map_persist:
                    review_word_map_persist[sr] = [i]
                else:
                    review_word_map_persist[sr]+=[i]
                    
                    
        review_content_persist[i] = ' '.join(file_words)
    
    with open(os.path.join(my_path, "indexes/review_content.pkl"), "wb") as review_content:
        pickle.dump(review_content_persist, review_content)
        review_content.close()
        
    with open(os.path.join(my_path, "indexes/review_word_map.pkl"), "wb") as review_word_map:
        pickle.dump(review_word_map_persist, review_word_map)
        review_word_map.close()
                

            

[nltk_data] Downloading package stopwords to C:\Users\Matteo
[nltk_data]     Cavalletti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
iindex_tf_idf_persist = {}

# Retriving precreated inverted indexes
with open(os.path.join(my_path, "indexes/iindex_tf_idf.pkl"), "rb") as iindex_tf_idf:
    iindex_tf_idf_persist = pickle.load(iindex_tf_idf)
    iindex_tf_idf.close()
    
if(len(iindex_tf_idf_persist.keys()) == 0):
    
    print("Inverted Indexes are being calculated")

    word_iindex = {}

    #Creating inverted index using tf-idf and consine similarity
    for word in review_word_map_persist:
        word_doc_list = review_word_map_persist[word]
        word_iindex[word] = []

        # Store indexes based on number of times a particular word is present in a given document
        for doc in word_doc_list:
            doc_content = review_content_persist[doc]
            word_iindex[word].append([doc, doc_content.split().count(word)])

    # Store indexes based on tf-idf
    docs_length = len(review_content_persist.keys())
    iindex_tf_idf_persist = word_iindex

    for key, word in iindex_tf_idf_persist.items():
        idf= math.log10( docs_length / len(word) )

        for elem in word:
            elem[1] = idf * elem[1]

    with open(os.path.join(my_path, "indexes/iindex_tf_idf.pkl"), "wb") as iindex_tf_idf:
        pickle.dump(iindex_tf_idf_persist, iindex_tf_idf)
        iindex_tf_idf.close()
        


In [26]:
# create dictionary of documents
# @TODO: Need to remove this dictionary if not used
dic_doc={}
for i in range(18259):
    dic_doc[r'docu_hw3\doc_'+str(i)+'.tsv']= i

In [27]:
word = input('Enter a search query: ')
    
def clean_input(w):
    w_list = []
    w = w.lower()
    w = tokenizer.tokenize(w)
    # Check if we need to do any other preprocessing to improve the efficiency of search results
    
    for r in w :
        if not r in stop_words:
            sr = ps.stem(r)
            if not  sr in w_list:
                w_list.append(sr)
    return w_list

word_list = clean_input(word)

print("Cleaned word: ", word_list)

list_doc_list = []

for w in word_list:
        doc_list = []
        
        if w in review_word_map_persist: 
            doc_list = review_word_map_persist[w]
            
        list_doc_list.append(doc_list)  
        
list_intersect = list_doc_list[0]

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

        
for docList in list_doc_list:
    list_intersect = intersection(list_intersect, docList)
    
results = []

list_intersect = list(set(list_intersect))
        
for doc in list_intersect[:10]:
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    
    results.append([title, description, city, url])


    
if(len(results)):
    print('Found ' + str(len(list_intersect))  + ' matching reviews to your query')
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results)
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))
 

Enter a search query: beautiful
Cleaned word:  ['beauti']
Found 3185 matching reviews to your query


Title,Description,City,URL
Unique Location! Alamo Heights - Designer Inspired,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. Amazing location - House conveniently located in quiet street, with beautiful seasoned trees, prestigious neighborhood and very close to the airport, 281, 410 loop and down-town area. Featuring an open floor plan, original hardwood floors, 3 bedrooms, 3 FULL bathrooms + an independent garden-TV room which can sleep 2 more European inspired kitchen and “top of the line” decor. Driveway can park 4 cars.",San Antonio,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX
The Hill Country Oasis,"Located steps from Texas' original hill country golf resort & spa, Tapatio Springs, this home boasts beautiful views & a warm inviting feel. This is the ultimate getaway for any avid golfer with 18 holes of premier hill country golf just seconds away! It's an easy walk to the driving range & chipping/putting green that can be seen from the back deck. Enjoy a nice dinner at home or walk to the resort & enjoy some fine dining, cocktails & live music without having to worry about driving home.",Boerne,https://www.airbnb.com/rooms/18597690?location=Center%20Point%2C%20TX
Friendly Private Room in َQuiet Neighborhood,"This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The bathroom is shared and supplies such as towels and shampoo are available. We are only some miles from Downtown, TCU, TCC, and Stockyards.",Fort Worth,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
Good Morning Ranch : Farmhouse on 10 acres,"Good Morning Ranch is a beautiful barn-style farmhouse that sits hilltop on 10 gorgeous acres only a few minutes from the town square with two rain-fed ponds and sweeping hill country views. It has 3 bedrooms, a sun-and-sleeping porch, balcony, 2 shaded outdoor porches, wood-burning fireplace, fire pit, grill, 18 foot ceilings, all new stainless steel appliances, a custom built farm table, vintage decor, and lots of space to explore.",Wimberley,https://www.airbnb.com/rooms/13155551?location=Canyon%20Lake%2C%20TX
Quiet Cozy Room,"Gated Estate in the heart of the DFW metro. Sets on 2.5 wooded acres. Home overlooks a beautiful pond. 8 -10 mins to DFW Airport and 25-35 mins to Love Field Airport. Located minutes away from Dallas & Fort Worth, American Airlines Center, Dallas museums, Deep Elum, AT&T Stadium, Global Life Park, Six Flags, Fort Worth Stock Yards, Texas Motor Speed Way, Grapevine Mills Mall, Gaylord Texan, Great Wolf Lodge, Check out our other listing we have at our home - FABEULESS HIDEAWAY",Euless,https://www.airbnb.com/rooms/4251773?location=Colleyville%2C%20TX
A Cozy Home with a Beautiful Nature Views,"This home is on the North Side of San Antonio, 3 minutes away from a great golfing course and a JW Marriott Resort and Spa. This is a great stopping place for a family and comes with all amenities from cable, internet to a laundry room and 2 bathrooms. This house is against a green belt so you can enjoy the back yard and nature.This area is safe and my neighbors are also great! Hope you enjoy your stay and more so enjoy San Antonio!",San Antonio,https://www.airbnb.com/rooms/19190311?location=Cibolo%2C%20TX
"Cozy cottage charm,right in Kerrville!","My place is close to downtown Kerrville. Beautiful pine ceilings and cozy fireplace. Cottage appeal in a unique and eclectic Kerrville neighborhood! Comes with screened porch and huge backyard. Washer dryer available also for guest's convenience. Great place to use as a home base for exploring Texas wine country nearby. I have a covered carport with 2 parking spaces included in the rental price-which is great to have if we get rain or bad weather, your car is protexted.",Kerrville,https://www.airbnb.com/rooms/14694333?location=Center%20Point%2C%20TX
Dallas Bungalow - White Rock Lake,"Stay in the heart of Dallas in this beautiful bungalow. Sleeps 2-3 with queen 

In [28]:
dict_qcos = {}
dict_norm = {}


#print("Search started")

for doc in list_intersect[:100]:
    
    num = 0
    
    #print("Current document ID: " + str(doc))
    
    for word in word_list:
        w_index = iindex_tf_idf_persist[word]
        w_i_len = len(w_index)
        for i in range(w_i_len):
            if w_index[i][0] == doc: 
                num +=  w_index[i][1]
                
    dict_qcos[doc]=num

    
    #denominator
    norm = 0
    for word in iindex_tf_idf_persist.values():
        for i in range(len(word)):
            if word[i][0] == doc:
                norm +=  word[i][1]**2
    
    dict_norm[doc]=math.sqrt(norm)

#print("Numerator and Denominator calculated")
    
for doc,num in dict_qcos.items():
    if dict_norm[doc] != 0:
        dict_qcos[doc] = num/(math.sqrt(len(word_list))*dict_norm[doc])

h = []
results = []

#print("Cosine similarity done")

In [29]:
h = []
results = []

for doc in dict_qcos.keys():
    
    with open(os.path.join(my_path, 'docu_hw3/doc_' + str(doc) + '.tsv'),encoding='utf8') as tsvfile:
         tsvreader = list(csv.reader(tsvfile, delimiter="\t"))
    
    title = tsvreader[0][7]        
    description = tsvreader[0][4]
    city = tsvreader[0][2]
    url = tsvreader[0][8]
    
    
    heapq.heappush(h,(dict_qcos[doc], title, description, city, url))

#print("Applying Heap")    
heapq._heapify_max(h)

for i in range(10):
    
    results.append(list(heapq.heappop(h)))
    #print(heapq.heappop(h))

    heapq._heapify_max(h)

#print("Applying Heap")  

#print(results)
results_formatted = []


for i in results:
    
    first = round(i.pop(0), 4)
    i.append(first) 
    #first = round(float(results[i].pop(0)), 4)
    #results[i] = results[i].append(first)
    results_formatted.append(i)


if(len(results_formatted)):
    display(HTML('<table border="1"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th><th>Score</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in results_formatted)
)))
else:
    display(HTML('<h1>No results found. Please try a different query</h1>'))

Title,Description,City,URL,Score
Beautiful Private Room/ Bathroom Awesome Location,Gorgeous Private room and bathroom in a beautiful home for super bowl weekend!,Richmond,https://www.airbnb.com/rooms/17014230?location=Cinco%20Ranch%2C%20TX,0.5821
Beautiful Apartment near downtowm,"Beautiful one bedroom apartment fully loaded available. All amenities include pool, grill, washer, dryer etc.",Austin,https://www.airbnb.com/rooms/15252117?location=Buda%2C%20TX,0.3982
2nd Floor Oasis in Beautiful Home,"Second floor accommodations in gorgeous Cape Cod style home. Large beautifully appointed bedroom with sitting chair, music player, private living area and private bathroom. Relax on the balcony overlooking the backyard.",Aubrey,https://www.airbnb.com/rooms/7478780?location=Aubrey%2C%20TX,0.3221
Romantic getaway,A beautiful 3/2 home 10 minutes from sea world and fiesta Texas . 30 min from Lackland AFB Great location also close to the river walk.You can feel right at home while you are vacationing in this beautiful city. Kitchen has granite countertops.Bathroom offer garden tub/shower. All bedroom have ceiling fan. Privacy and covered back porch. This home has everything you need to spend a nice romantic get away vacation in San Antonio. Requires 2 minimum night stay,San Antonio,https://www.airbnb.com/rooms/19142171?location=Castroville%2C%20TX,0.2369
Stay and Play on the Comal River,Beautiful 2 bedroom 2 1/2 bath condo on the Comal River. A traditional family vacation spot where fond memories can be made on the beautiful blue Comal River.,New Braunfels,https://www.airbnb.com/rooms/12515991?location=Cibolo%2C%20TX,0.2145
Luxurious suite +Parking +Rooftop Pool @Downtown,"Click on my profile to check out my 2 other Downtown Dallas apartments: Another 1-bedroom apartment & a 2-bedrooms apartment You are looking at the beautiful high-rise suite right at the heart of Downtown Dallas. You can certainly walk to almost everything you need from high-end restaurants to parks, shopping, and tourist destinations. It offers a fully equipped kitchen, stainless steel appliances, hardwood flooring, in-home washer and dryer, beautiful pool, fitness center, and sundeck.",Dallas,https://www.airbnb.com/rooms/17531497?location=Brazos%20River%2C%20TX,0.2092
"Elegant apartment by Six Flags, La Cantera","Located just north of the beautiful Dominion. This area is within a few miles of Six Flags Fiesta Texas, The Rim, La Cantera and so much more. Within walking distance of The Grill at Leon Springs and so many more great attractions. This place is beautifully decorated and very comfortable. Whether you're relaxing poolside or enjoying the comfortable apartment, you'll love the serene settings",San Antonio,https://www.airbnb.com/rooms/19332070?location=Boerne%2C%20TX,0.2024
MODERN HOME with SWIMMING POOL - The Woodlands,"This family home is located in Shenandoah,towards the outskirts of The Woodlands.The Woodlands is unique in its beauty, diversity, convenience and comfort.You can experience all of its beauty on your own in the comfort of your own personal vacation retreat.The house may look small from the outside, but many guests feel at home due to the modern décor. This modern home is perfect for weekend getaways or longer work related stays and for families.",Shenandoah,https://www.airbnb.com/rooms/19264952?location=Cleveland%2C%20TX,0.1866
Beautiful bedroom with private bath and closet,"A lovely bedroom with private bath and closet on 1st floor of a beautiful townhouse in Las Colinas, Irving. Just 10 mins from DFW and 15 mins to downtown Dallas. Peaceful and safe neighborhood. Our townhouse was built in 2016 and everything inside is brand new. Only 1 bedroom in 1st so guests can enjoy spacious area and privacy. Living area and kitchen on 2nd floor with modern decoration. You are welcome to use 2nd floor but please keep everything clean and tidy.",Irving,https://www.airbnb.com/rooms/19409762?location=Carrollton%2C%20TX,0.1858
Charming 1/BR in a great location